# Missing HYSPLIT runs/files?

The idea of this script is to simply look at the folders where you have saved your runs. Create a dataframe which lists the missing runs (sometimes you have to end a loop and you forget to generate the rest e.g.). Once the dataframe of the missing datetimes is created. Just need to loop through them and generate HYSPLIT files for them.  

this works with files with the name: 
GDAS1apr0250spring2013040100

In [1]:
import sys
import numpy as np
import datetime
from datetime import timedelta  
from itertools import product
import glob
import pandas as pd
from tqdm import tqdm, trange
import os

sys.path.append(r'C:\Users\DominicHeslinRees\Documents\ACP_2023\scripts\HYSPLIT-processing')
import HYSPLIT_processing as HYprocess

%load_ext autoreload
%autoreload 2

In [5]:
path_HYSPLIT_files = "D:\\HYSPLIT_runs_ensemble\\"

dict_year_to_HYSPLIT_folder = {}
for year in list(np.arange(2002, 2024, 1)):
    dict_year_to_HYSPLIT_folder[year] = str(year)
print(dict_year_to_HYSPLIT_folder)

{2002: '2002', 2003: '2003', 2004: '2004', 2005: '2005', 2006: '2006', 2007: '2007', 2008: '2008', 2009: '2009', 2010: '2010', 2011: '2011', 2012: '2012', 2013: '2013', 2014: '2014', 2015: '2015', 2016: '2016', 2017: '2017', 2018: '2018', 2019: '2019', 2020: '2020', 2021: '2021', 2022: '2022', 2023: '2023'}


In [6]:
year = 2023
files = glob.glob(path_HYSPLIT_files+str(year)+"\\GDAS*")
print(files[0])

D:\HYSPLIT_runs_ensemble\2023\GDAS1mar0250spring2023032705


In [7]:
def sample_filter_name_to_data(sample_name):
    """get digits in strings"""
    sample_name_unaltered = sample_name
    sample_name = sample_name[-10:]
    try:
        sample_name = pd.to_datetime(sample_name, format='%Y%m%d%H')
    except:
        print("error:")
        print(sample_name)
        print(sample_name_unaltered)
    return sample_name    

In [8]:
sample_filter_name_to_data(files[0])

Timestamp('2023-03-27 05:00:00')

In [12]:
def fraction_done(list_files, year):
    datetimes = [sample_filter_name_to_data(x) for x in list_files]
    df_datetimes = pd.DataFrame(index=datetimes)
    df_datetimes.index = pd.to_datetime(df_datetimes.index)
    df_datetimes['month'] = df_datetimes.index.month
    months_counted = df_datetimes['month'].value_counts()
    
    df_months = pd.DataFrame(months_counted)   
    rest = list(set(np.arange(1, 13, 1))-set(df_months.index)) #rest of the months not run yet

    for i in rest:
        df_months.loc[str(i)] = np.nan    
    df_months.index = df_months.index.astype(int)    
        
    if year in [2004, 2008, 2012, 2016, 2020]: #leap year different dictionary of days
        print("leap year:")
        dict_months_to_days = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    if year not in [2004, 2008, 2012, 2016, 2020]:
        print("normal year:")
        dict_months_to_days = {1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
    
    df_months['potential_days'] = df_months.index.map(dict_months_to_days)
    df_months['potential_data_points'] = df_months['potential_days']*24
    df_months['fraction'] = df_months['month']/df_months['potential_data_points']
    df_months = df_months.sort_index()
    fraction = df_months['month'].sum() / df_months['potential_data_points'].sum()
    df_months = df_months.rename(columns={'month':'hours_in_month_generated'})
    df_months = df_months.rename(columns={'potential_data_points':'potential_hourly_data_points'})
    print("fraction produced: "+str(fraction))
    df_months = df_months.sort_index()
    return df_months

In [17]:
year=2023

In [18]:
files = glob.glob(path_HYSPLIT_files+str(year)+"\\GDAS*")
fraction_done(files, year=year)

normal year:
fraction produced: 0.8711187214611872


,hours_in_month_generated,potential_days,potential_hourly_data_points,fraction
1,744.0,31,744,1.000000
2,672.0,28,672,1.000000
3,744.0,31,744,1.000000
4,720.0,30,720,1.000000
5,744.0,31,744,1.000000
6,720.0,30,720,1.000000
7,744.0,31,744,1.000000
8,744.0,31,744,1.000000
9,720.0,30,720,1.000000
10,744.0,31,744,1.000000


In [19]:
def generate_datetimes(year):
    start = datetime.datetime.strptime("01-01-"+str(year), "%d-%m-%Y")
    end = datetime.datetime.strptime("31-12-"+str(year), "%d-%m-%Y")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
    dates = [x.strftime("%d-%m-%Y") for x in date_generated]   
    return dates

In [20]:
dates = generate_datetimes(year)

In [21]:
def add_times(dates):
    times = [timedelta(hours=int(i)) for i in np.arange(0, 24, 1)]
    times = [str(datetime.datetime.strptime(str(x),'%H:%M:%S').time()) for x in times]
    datetimes = list(product(dates,times))
    datetimes = [pd.to_datetime(x[0] +' '+ x[1]) for x in datetimes]
    return datetimes

In [22]:
path_HYSPLIT_files

'D:\\HYSPLIT_runs_ensemble\\'

In [23]:
dict_year_to_HYSPLIT_folder

{2002: '2002',
 2003: '2003',
 2004: '2004',
 2005: '2005',
 2006: '2006',
 2007: '2007',
 2008: '2008',
 2009: '2009',
 2010: '2010',
 2011: '2011',
 2012: '2012',
 2013: '2013',
 2014: '2014',
 2015: '2015',
 2016: '2016',
 2017: '2017',
 2018: '2018',
 2019: '2019',
 2020: '2020',
 2021: '2021',
 2022: '2022',
 2023: '2023'}

In [24]:
years = 2023
list_of_ = HYprocess.find_HYSPLIT_files_per_year(years, path_HYSPLIT_files, dict_year_to_HYSPLIT_folder, 
                                      prefix='GDAS', inputyr_2digits=False)
print(len(list_of_))

Year: 2023
Path: D:\HYSPLIT_runs_ensemble\2023\
2023
number of files: 7631
inputyr_2digits=False i.e. str(year)
added day and month now*
D:\HYSPLIT_runs_ensemble\2023\GDAS*2023*
Number of HYSPLIT files for 2023: 7621
7621


In [25]:
def generate_df_missing_datetimes(years, inpath, prefix='GDAS'):
    dict_year_to_df = {}
    dict_year_to_missing_df = {}
    for year in years: 
        print("year: "+str(year))   
        list_of_HYSPLIT_files_per_year = HYprocess.find_HYSPLIT_files_per_year(year, inpath, 
                                                                   dict_year_to_HYSPLIT_folder, 
                                                                   prefix) #list of HYSPLIT files
        if len(list_of_HYSPLIT_files_per_year) > 0:
            list_files = list_of_HYSPLIT_files_per_year.copy() #copy for later
            df_months = fraction_done(list_files, year)
            dict_year_to_df[year] = df_months 

            datetimes = [sample_filter_name_to_data(x) for x in list_files]

            dates = generate_datetimes(year=year)    
            datetimes_potential = add_times(dates)

            missing_datetimes_list = set(datetimes_potential).difference(set(datetimes))
            #print(missing_datetimes_list)
            print("number of missing datetimes: "+str(len(missing_datetimes_list)))   

            if len(missing_datetimes_list) > 0:
                df_missing_datetimes = pd.DataFrame(index=missing_datetimes_list)
                dict_year_to_missing_df[year] = df_missing_datetimes 
    
    if len(dict_year_to_missing_df) > 0:    
        df_all_missing = pd.concat(dict_year_to_missing_df)
        df_all_missing = df_all_missing.reset_index()
        df_all_missing.columns = ['year', 'datetime']
        df_all_missing = df_all_missing.sort_values(['datetime'])
        
    if len(dict_year_to_missing_df) == 0:
        print("all good")
        df_all_missing = pd.DataFrame(columns=['year', 'datetime'])
        dict_year_to_missing_df
    
    return df_all_missing, dict_year_to_df

In [26]:
print(path_HYSPLIT_files)
df_all_missing, dict_year_to_df = generate_df_missing_datetimes(years = list(np.arange(2002, 2024, 1)),
                                              inpath=path_HYSPLIT_files, prefix='GDAS')

D:\HYSPLIT_runs_ensemble\
year: 2002
Year: 2002
Path: D:\HYSPLIT_runs_ensemble\2002\
2002
year has a length of zero: 
Number of HYSPLIT files for 2002: 0
year: 2003
Year: 2003
Path: D:\HYSPLIT_runs_ensemble\2003\
2003
year has a length of zero: 
Number of HYSPLIT files for 2003: 0
year: 2004
Year: 2004
Path: D:\HYSPLIT_runs_ensemble\2004\
2004
year has a length of zero: 
Number of HYSPLIT files for 2004: 0
year: 2005
Year: 2005
Path: D:\HYSPLIT_runs_ensemble\2005\
2005
number of files: 8736
inputyr_2digits=False i.e. str(year)
added day and month now*
D:\HYSPLIT_runs_ensemble\2005\GDAS*2005*
Number of HYSPLIT files for 2005: 8724
normal year:
fraction produced: 0.9958904109589041
number of missing datetimes: 12
year: 2006
Year: 2006
Path: D:\HYSPLIT_runs_ensemble\2006\
2006
number of files: 8773
inputyr_2digits=False i.e. str(year)
added day and month now*
D:\HYSPLIT_runs_ensemble\2006\GDAS*2006*
Number of HYSPLIT files for 2006: 8749
normal year:
fraction produced: 0.9987442922374429


In [27]:
df_all_missing.year.unique()

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023], dtype=int64)

In [28]:
df_all_missing.head(5)

,year,datetime
3,2005,2005-01-20 05:00:00
10,2005,2005-02-20 05:00:00
4,2005,2005-03-20 05:00:00
2,2005,2005-04-20 05:00:00
1,2005,2005-05-20 05:00:00


In [29]:
len(df_all_missing)

1427

In [30]:
def get_memory(filename):
    memory = os.stat(filename).st_size
    #print(memory)
    return memory

In [31]:
def get_list_of_files_with_too_little_memory(files, memory_threshold, year=None):
    files_too_little_memory = []
    if year is not None:
        files = glob.glob(path_HYSPLIT_files+str(year)+"\\GDAS*")       
    for file in files:
        memory = get_memory(filename=file)
        if memory < memory_threshold:
            #print(memory)
            #print(file)
            files_too_little_memory.append(file)
    return files_too_little_memory

In [32]:
files_too_little_memory = get_list_of_files_with_too_little_memory(files=None, memory_threshold=8000, year=2017)

In [33]:
def generate_df_missing_datetimes_memory(years, inpath, prefix='GDAS'):
    dict_year_to_df = {}
    dict_year_to_missing_df = {}
    for year in years: 
        print("year: "+str(year))   
        list_of_HYSPLIT_files_per_year = HYprocess.find_HYSPLIT_files_per_year(year, inpath, 
                                                                   dict_year_to_HYSPLIT_folder, 
                                                                   prefix) #list of HYSPLIT files
        if len(list_of_HYSPLIT_files_per_year) > 0:
            list_files = list_of_HYSPLIT_files_per_year.copy() #copy for later
            df_months = fraction_done(list_files, year)
            dict_year_to_df[year] = df_months 
            
            files_too_little_memory = get_list_of_files_with_too_little_memory(list_files, memory_threshold=8000)
    
            missing_datetimes_list = [sample_filter_name_to_data(x) for x in files_too_little_memory]

            print("number of datetimes with too little memory: "+str(len(files_too_little_memory )))   

            if len(files_too_little_memory) > 0:
                print("more than one")
                df_missing_datetimes = pd.DataFrame(index=missing_datetimes_list)
                dict_year_to_missing_df[year] = df_missing_datetimes  
            if len(files_too_little_memory) == 0:
                dict_year_to_missing_df[year] = pd.DataFrame()
                pass
                
    if len(df_missing_datetimes) > 0:
        df_all_missing = pd.concat(dict_year_to_missing_df)
        df_all_missing = df_all_missing.reset_index()
        df_all_missing.columns = ['year', 'datetime']
        df_all_missing = df_all_missing.sort_values(['datetime'])
            
    if len(df_missing_datetimes) == 0:
        print("all good")
        df_all_missing = pd.DataFrame(columns=['year', 'datetime'])
        
    return df_all_missing, dict_year_to_df

In [34]:
df_all_missing_memory, dict_year_to_df_memory = generate_df_missing_datetimes_memory(years=list(np.arange(2002, 2024, 1)), 
                                                                                     inpath=path_HYSPLIT_files,
                                                                                     prefix='GDAS')

year: 2002
Year: 2002
Path: D:\HYSPLIT_runs_ensemble\2002\
2002
year has a length of zero: 
Number of HYSPLIT files for 2002: 0
year: 2003
Year: 2003
Path: D:\HYSPLIT_runs_ensemble\2003\
2003
year has a length of zero: 
Number of HYSPLIT files for 2003: 0
year: 2004
Year: 2004
Path: D:\HYSPLIT_runs_ensemble\2004\
2004
year has a length of zero: 
Number of HYSPLIT files for 2004: 0
year: 2005
Year: 2005
Path: D:\HYSPLIT_runs_ensemble\2005\
2005
number of files: 8736
inputyr_2digits=False i.e. str(year)
added day and month now*
D:\HYSPLIT_runs_ensemble\2005\GDAS*2005*
Number of HYSPLIT files for 2005: 8724
normal year:
fraction produced: 0.9958904109589041
number of datetimes with too little memory: 89
more than one
year: 2006
Year: 2006
Path: D:\HYSPLIT_runs_ensemble\2006\
2006
number of files: 8773
inputyr_2digits=False i.e. str(year)
added day and month now*
D:\HYSPLIT_runs_ensemble\2006\GDAS*2006*
Number of HYSPLIT files for 2006: 8749
normal year:
fraction produced: 0.99874429223744

In [35]:
df_all_missing_memory.head(5)

,year,datetime
29,2005,2005-09-07 16:00:00
17,2005,2005-09-07 17:00:00
11,2005,2005-09-07 18:00:00
66,2005,2005-09-07 19:00:00
30,2005,2005-09-07 20:00:00


In [36]:
len(df_all_missing_memory)

316

In [37]:
years = df_all_missing.year.unique()
print(years)
for year in years:
    print(year)
    print(dict_year_to_df[year])

[2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018
 2019 2020 2021 2022 2023]
2005
    hours_in_month_generated  potential_days  potential_hourly_data_points  \
1                        743              31                           744   
2                        671              28                           672   
3                        743              31                           744   
4                        719              30                           720   
5                        743              31                           744   
6                        719              30                           720   
7                        743              31                           744   
8                        743              31                           744   
9                        719              30                           720   
10                       743              31                           744   
11                       719           

# Running: 

In [38]:
import os
os.environ["PROJ_LIB"] = "C:\\Utilities\\Python\\Anaconda\\Library\\share"; #fixr
import pysplit

In [39]:
def add_front_0(number):
    #print(f'{number:02d}')
    #f"{number:0>2}"
    return f'{number:02d}'

In [40]:
def generate_based_on_df(df_all_missing, meteo_dir = r'D:/GDAS_met_files', storage_dir=r'D:/HYSPLIT_runs_ensemble/'):
    altitude = 250
    location = (78.906, 11.888) 
    runtime = -240
    altitudes = [250]

    basename = 'GDAS1'
    working_dir = r'C:/hysplit/working/'
    hysplit="C:/hysplit/exec/hyts_ens" #ensemble

    df_all_missing['month'] = df_all_missing.datetime.dt.month 
    df_all_missing['day'] = df_all_missing.datetime.dt.day 
    df_all_missing['hour'] = df_all_missing.datetime.dt.hour

    for i in tqdm(range(len(df_all_missing))):
        year = df_all_missing.loc[i, 'year']
        month = df_all_missing.loc[i, 'month']
        day = int(df_all_missing.loc[i, 'day'])
        hour = df_all_missing.loc[i, 'hour']

        years = [year] #missing 1,6-12 do for 2 to see if same    
        months = [month] #looped for the 11, so will no 
        hours = [hour] #fixed

        datetime = pd.to_datetime(str(years[0])+str(add_front_0(int(months[0])))+str(add_front_0(int(day)))+str(add_front_0(int(hours[0]))),
                                  format='%Y%m%d%H')
        print("Datetime to process:")
        print(datetime)       

        storage_dir_year = str(storage_dir)+str(year) 

        pysplit.generate_bulktraj(basename, working_dir, storage_dir_year, meteo_dir, years,
                          months, hours, altitudes, location, runtime,
                          meteoyr_2digits=True, outputyr_2digits=False,
                          monthslice=slice(day-1, day, 1), meteo_bookends=([4, 5], [1]),
                          get_reverse=False, get_clipped=False,
                          hysplit=hysplit)

In [41]:
generate_based_on_df(df_all_missing_memory, meteo_dir = r'D:/GDAS_met_files', 
                     storage_dir=r'D:/HYSPLIT_runs_ensemble/')

  0%|                                                                                          | 0/316 [00:00<?, ?it/s]

Datetime to process:
2005-09-19 23:00:00


  1%|▊                                                                                 | 3/316 [00:03<04:42,  1.11it/s]

Datetime to process:
2005-09-20 11:00:00
Datetime to process:
2005-09-20 04:00:00


  1%|█                                                                                 | 4/316 [00:03<03:03,  1.70it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████████▏| 313/316 [01:35<00:00, 11.81it/s]

Datetime to process:
2008-05-14 21:00:00
Datetime to process:
2008-05-14 22:00:00
Datetime to process:
2008-05-14 23:00:00


100%|███████████████████████████████████████████████████████████████████████████████▋| 315/316 [01:35<00:00, 12.02it/s]

Datetime to process:
2008-05-15 00:00:00
Datetime to process:
2023-11-23 06:00:00


100%|████████████████████████████████████████████████████████████████████████████████| 316/316 [01:37<00:00,  3.23it/s]
